In [ ]:
from xradio.correlated_data import convert_msv2_to_processing_set

#convert_msv2_to_processing_set('ngc3256_line_target.ms','ngc3256_line_target.vis.zarr',partition_scheme=[])
convert_msv2_to_processing_set('twhya_calibrated.ms','twhya_calibrated.vis.zarr')

In [ ]:
from xradio.correlated_data import open_processing_set

ps = open_processing_set('twhya_calibrated.vis.zarr')
ps.summary()

In [ ]:
ps = ps.sel(intents=["OBSERVE_TARGET#ON_SOURCE"])
ps.summary()

In [1]:
# import dask
# dask.config.set(scheduler='synchronous')

from toolviper.dask.client import local_client
viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-09-20 11:21:41,209]     INFO   toolviper:  Module path: /Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/toolviper 
[2024-09-20 11:21:41,212]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 
[2024-09-20 11:21:42,081]     INFO      client:  Client <MenrvaClient: 'tcp://127.0.0.1:63223' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63223,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:63235,Total threads: 1
Dashboard: http://127.0.0.1:63238/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:63226,


In [2]:
import numpy as np
import xarray as xr
import dask
from graphviper.graph_tools.coordinate_utils import make_parallel_coord, make_frequency_coord
from xradio.correlated_data import open_processing_set
from graphviper.graph_tools import map, reduce
from astroviper._domain._image._moment_max import _moment_max
from astroviper.imaging._utils import _make_image_single_field
from toolviper.utils.display import dict_to_html
from IPython.display import HTML, display


#ps_store = "Antennae_North.cal.lsrk.split.py39.v3.vis.zarr"
ps_store = 'twhya_calibrated.vis.zarr'
intents = ["OBSERVE_TARGET#ON_SOURCE"]
ps = open_processing_set(ps_store, intents=intents)
ms_xds = ps.get(0)

grid_params = {}
grid_params["chan_mode"] = "cube"
grid_params["image_size"] = [250, 250]
grid_params["cell_size"] = np.array([-0.1, 0.1]) * np.pi / (180 * 3600)
grid_params["fft_padding"] = 1.0
grid_params["phase_direction"] = ps.get(0).VISIBILITY.field_and_source_xds.FIELD_PHASE_CENTER # [ 3.14980724 -0.32929961]

input_params = {}
input_params["grid_params"] = grid_params
input_params["to_disk"] = False
input_params["input_data_store"]=ps_store
input_params["polarization"] = ms_xds.polarization
input_params["time"] = [0]
input_params["frequency"] = None
input_params['data_group'] = "base"

parallel_coords = {}
parallel_coords["frequency"] = make_parallel_coord(ps.get_ps_freq_axis(), n_chunks=8)

from graphviper.graph_tools.coordinate_utils import interpolate_data_coords_onto_parallel_coords
node_task_data_mapping = interpolate_data_coords_onto_parallel_coords(parallel_coords, ps)
#display(HTML(dict_to_html(node_task_data_mapping)))

import time
graph = map(
    input_data=ps,
    node_task_data_mapping=node_task_data_mapping,
    node_task=_make_image_single_field,
    input_params=input_params,
    in_memory_compute=False
)

from graphviper.graph_tools import generate_dask_workflow
# dask_graph = generate_dask_workflow(graph)


input_params = {}
viper_graph = reduce(graph, _moment_max, input_params)


dask_graph = generate_dask_workflow(viper_graph)

img_xds = dask.compute(dask_graph)[0]

2024-09-20 11:21:52,515 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:63234
Traceback (most recent call last):
  File "/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/distributed/comm/tcp.py", line 227, in read
    frames_nosplit = await read_bytes_rw(stream, frames_nosplit_nbytes)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/distributed/comm/tcp.py", line 366, in read_bytes_rw
    actual = await stream.read_into(chunk)  # type: ignore[arg-type]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jsteeb/miniforge3/envs/zinc/lib/python3.12/site-packages/distributed/worker.py", line 2056, in gather_dep
    response = await get_data_from_worker(
               ^^^^^^^^^^^^

In [ ]:
img_xds

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
img_xds.SKY.isel(polarization=0, frequency=0, dummy=0).plot.imshow(cmap='viridis',vmin=0.0)
plt.figure()
img_xds.PRIMARY_BEAM.isel(polarization=0, frequency=0, dummy=0).plot()
plt.figure()
img_xds.POINT_SPREAD_FUNCTION.isel(polarization=0, frequency=0, dummy=0).plot()


In [ ]:
ps.get(0).baseline_antenna1_name.values

In [ ]:
ps.get(0).baseline_antenna2_name.values

In [ ]:
import xarray as xr
import numpy as np

# Assuming `ds` is your xarray Dataset
ds = xr.Dataset({
    'baseline_antenna1_name': ('x', ['DV04_J505', 'DV04_J505', 'DV04_J505', 'DV04_J505', 'DV04_J505',
       'DV04_J505', 'DV04_J505', 'DV06_T704', 'DV06_T704', 'DV06_T704',
       'DV06_T704', 'DV06_T704', 'DV06_T704', 'DV07_J510', 'DV07_J510',
       'DV07_J510', 'DV07_J510', 'DV07_J510', 'DV08_T703', 'DV08_T703',
       'DV08_T703', 'DV08_T703', 'DV09_N602', 'DV09_N602', 'DV09_N602',
       'PM02_T701', 'PM02_T701', 'PM03_J504']),
    'baseline_antenna2_name': ('x', ['DV04_J505', 'DV06_T704', 'DV07_J510', 'DV08_T703', 'DV09_N602',
       'PM02_T701', 'PM03_J504', 'DV06_T704', 'DV07_J510', 'DV08_T703',
       'DV09_N602', 'PM02_T701', 'PM03_J504', 'DV07_J510', 'DV08_T703',
       'DV09_N602', 'PM02_T701', 'PM03_J504', 'DV08_T703', 'DV09_N602',
       'PM02_T701', 'PM03_J504', 'DV09_N602', 'PM02_T701', 'PM03_J504',
       'PM02_T701', 'PM03_J504', 'PM03_J504'])
})

# Create a mask where baseline_antenna1_name does not equal baseline_antenna2_name
mask = ds['baseline_antenna1_name'] != ds['baseline_antenna2_name']

# Apply the mask to the Dataset
filtered_ds = ds.where(mask, drop=True)

print(filtered_ds)